In [21]:
import pandas as pd
import numpy as np

### 날짜/시간별 지하철 승객

In [162]:
ef = pd.read_csv('지하철승객.csv')

In [163]:
ef.head()

,날짜,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,2019-01-01,10283,NaN,NaN,NaN,NaN,62190,115735,99437,139613,...,312610,325063,341955,352824,324252,265904,241062,231635,202026,109786
1,2019-01-02,60657,NaN,NaN,NaN,NaN,75575,229632,667102,1211838,...,444328,477788,523794,683327,1160405,809542,481659,427546,366026,196117
2,2019-01-03,68625,NaN,NaN,NaN,NaN,78420,225188,653359,1237798,...,466252,499921,546788,708854,1160203,811648,495906,461639,413884,225704
3,2019-01-04,103120,NaN,NaN,NaN,NaN,79265,221692,637510,1230152,...,481249,523817,589647,777479,1251689,882007,512768,455779,442913,281368
4,2019-01-05,20652,NaN,NaN,NaN,NaN,67812,138598,180598,318055,...,581275,579735,625712,665117,579595,428976,368067,364591,336160,204776


### 날씨 데이터에서 일시 열만 빼기

wf = pd.read_csv('날씨.csv')
rf = wf # rf는 나중에 있을 merge를 위해 날씨 데이터를 따로 뽑아 놓았습니다.
wf = wf.drop(['기온(°C)','풍속(m/s)', '습도(%)', '현지기압(hPa)',
       '전운량(10분위)'], axis=1)
wf.head()

### 시간별 지하철 승객의 행을 날씨데이터에서 뽑은 열에 붙이기

In [165]:
ef.loc[ef['날짜'] == single_date]
# 조건 걸고 row 하나 뽑아옴

,날짜,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
364,2019-12-31,216799,NaN,NaN,NaN,NaN,70162,193561,530880,1024180,...,544110,620213,702612,819956,962823,653786,465354,415102,394194,229730


In [166]:
ef.loc[ef['날짜'] == single_date].T
# Transpose 해서 인덱스와 컬럼을 서로 위치교환(돌림)

,364
날짜,2019-12-31
0,216799
1,NaN
2,NaN
3,NaN
4,NaN
5,70162
6,193561
7,530880
8,1024180


In [167]:
ef.loc[ef['날짜'] == single_date].T[1:]
# 맨 위에 원래 날짜가 있었는데 슬라이싱으로 날짜를 뺌

,364
0,216799
1,NaN
2,NaN
3,NaN
4,NaN
5,70162
6,193561
7,530880
8,1024180
9,604786


In [168]:
ef.loc[ef['날짜'] == single_date].T[1:].reset_index()
# reset_index 이용해서 index에 있던 시간정보를 컬럼으로 올려주었습니다

,index,364
0,0,216799
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
5,5,70162
6,6,193561
7,7,530880
8,8,1024180
9,9,604786


In [171]:
wf['date'] = wf['일시'].apply(lambda x : x[:10])
datetimer = wf['일시'].apply(lambda x : pd.to_datetime(x))
wf['hour'] = datetimer.apply(lambda x : x.hour)

In [185]:
wf

,일시
0,2019-01-01 0:00
1,2019-01-01 1:00
2,2019-01-01 2:00
3,2019-01-01 3:00
4,2019-01-01 4:00
...,...
8732,2019-12-30 20:00
8733,2019-12-30 21:00
8734,2019-12-30 22:00
8735,2019-12-30 23:00


In [174]:
all_data = pd.DataFrame()
for single_date in np.unique(wf['date']):
    one_day = ef.loc[ef['날짜'] == single_date].T[1:].reset_index()
    
    one_day.columns = ['hour', '인원수']
    one_day['hour'] = one_day['hour'].apply(lambda x : int(int(x)))
    one_day = one_day.sort_values(by='hour')
    wf_one_day = wf.loc[wf['date'] == single_date]
    concat_ed = pd.merge(wf_one_day, one_day, how='inner', on='hour')
    all_data = pd.concat([all_data, concat_ed])
all_data.head()

,일시,date,hour,인원수
0,2019-01-01 0:00,2019-01-01,0,10283
1,2019-01-01 1:00,2019-01-01,1,NaN
2,2019-01-01 2:00,2019-01-01,2,NaN
3,2019-01-01 3:00,2019-01-01,3,NaN
4,2019-01-01 4:00,2019-01-01,4,NaN


### 날씨와 지하철 승객 수 Merge 후 파일 저장하기

In [179]:
all_data.to_csv('subway.csv', index=False, encoding='utf-8-sig')

In [187]:
df = all_data

In [189]:
df = pd.merge(rf,df).head()

In [191]:
df.to_csv('weather_subway.csv',index=False, encoding='utf-8-sig')